# <span style="color:seagreen">02-DAPI_max_projection</span>

In this notebook we will guide you through performing max projection of the DAPI channel. This max-projection is then used to create nuclear cell masks. Nuclear masks are then used to identify transcription sites. Note to perform the steps in this notebook, we will first need to create cell masks using Cellpose. 

## 2.0 - Load libraries

In [ ]:
import numpy as np
from glob import glob
from pathlib import Path
import re
from skimage import io
import matplotlib.pyplot as plt
from bigfish.detection import detect_spots
from bigfish.stack import remove_background_gaussian
from bigfish.stack import get_in_focus_indices
from bigfish.stack import compute_focus
from scipy.signal import savgol_filter
import napari
import seaborn as sns

import sys
sys.path.append('../')

from src.misc import group_experiments, load_data, find_high_density_patch, find_in_focus_indices

## 2.1 - Load data 

To perform zprojection of the in-focus layers a cell mask is required, as this is used to compute the in-focus layers in area with high cell density. Make sure you have created a cell mask an this mask can be found in the Masks folder, according to the format: "strain_mRNAs_condition_DIC_fovid_seg.tif".

In [ ]:
root_dir = '../Data/restructured_data/replicate1'
experiments = group_experiments(root_dir)
experiments_to_process=list(experiments.keys())
print(experiments_to_process)

Check if the cell mask of the selected image is loaded below. The format for the cell maks to be recognized is the name of the  corresponding DIC photo and ends with "_seg.tif".

In [ ]:
# select strain_mRNAs_condition and fovid combination 
identifier = 'cbk1_CLB2Q670HWP1CAL610_SPIDER37'
fovid= 6

experiments[identifier][fovid]

## 2.2 - Perform Zprojection

Perform Zprojection of the in-focus layers of the loaded image for which cell masks are available.

In [ ]:
#select a patch to calculate the in-focus layers.
patch_size = (500, 500)

In [ ]:
for identifier in experiments_to_process:
    fovs = experiments[identifier]
    
    for fov, paths, in fovs.items():
        print(f'processing {identifier=}, {fov=}')
        data = load_data(paths)
        
        #choose channel to perform maxprojection on
        channel='DAPI'
        
        process = True
        # check if all files required for this step have been loaded
        for entry in ['cell_mask', channel]:
            if data.get(entry) is None:
                print(f'{identifier=}, {fov=}, {entry=} could not be found')
                print(f'skipping {identifier=}, {fov=}!')
                process=False
        if process:
            #load channel.
            RNAs = data.get(channel)    

            mask = data.get('cell_mask')    
            #select high density patch.
            selected_patch = find_high_density_patch(mask, patch_size=patch_size)

            img_patch = RNAs[:,
                selected_patch[0]:selected_patch[0] + patch_size[0],
                selected_patch[1]:selected_patch[1] + patch_size[1]
            ]


            #compute in-focus layers
            focus = compute_focus(img_patch)
            projected_focus = np.max(focus, axis=(1, 2))

            #find in-focus layers
            projected_focus_smoothed = savgol_filter(projected_focus, 30, 2, 0)
            ifx_1, ifx_2 = find_in_focus_indices(projected_focus_smoothed)
            
            Zprojection_name=f"MAX_{Path(paths['DAPI']).name}"

            #perform max-projection
            DAPI_maxed=np.amax(RNAs[ifx_1:ifx_2, ...],axis=0)

            #save max projection
            io.imsave(Path(root_dir,'Zprojection',Zprojection_name),DAPI_maxed)

## 2.3 - Labelling of nuclear masks based on Zprojection

Perform labelling (nuclear mask creation) of all produced zprojection DAPI images. Here, we will use a produced zprojection to create a single nuclear masks image. These nuclear masks are used identify transcription sites during spot detection.

In [ ]:
Zprojection_paths=sorted(glob(str(Path(root_dir,'Zprojection',f'*.tif'))))

In [ ]:
import stackview
from scipy.ndimage import label as nlabel

#selecting images
i=0

zprojection=io.imread(Zprojection_paths[i])

#choose value between 0 and 100 to determnine DAPI signal from background.
threshold=40

#threshold image
image =(zprojection*(100/zprojection.max()))
seeds=image>threshold
labels,_=nlabel(seeds)

#create stackview curtain
stackview.curtain(zprojection, labels, continuous_update=True, alpha=0.5)

In [ ]:
#save resulting mask
nuclear_masks_name=f'{Path(Zprojection_paths[i]).stem[4:]}_seg.tif'
    

io.imsave(Path(Path(Zprojection_paths[i]).parents[1], 'Masks',nuclear_masks_name), labels)

## 2.4 - Batch labelling of nuclear masks 

In [ ]:
for zprojection_path in Zprojection_paths:
    
    zprojection=io.imread(zprojection_path)
    threshold=50
    image =(zprojection*(100/zprojection.max()))
    seeds=image>threshold
    labels,_=nlabel(seeds)
    
    nuclear_masks_name=f'{Path(zprojection_path).stem[4:]}_seg.tif'
    

    io.imsave(Path(Path(zprojection_path).parents[1], 'Masks',nuclear_masks_name), labels)

## 2.5 - Check and correct DAPI Mask corrections in Napari

Now that we have created nuclear masks these can be loaded an edited if necessary using Napari.

In [ ]:
root_dir = '../data/restructured_data/replicate1'
experiments = group_experiments(root_dir)

In [ ]:
experiments[identifier]

In [ ]:
identifier='cbk1_CLB2Q670HWP1CAL610_SPIDER37'
fov=6

viewer=napari.Viewer()

fov = experiments[identifier][fov]
 
        
#load images
DAPI = io.imread(fov['DAPI'])
nuclear_mask = io.imread(fov['nuclear_mask'])

viewer.add_image(DAPI)

viewer.add_labels(nuclear_mask, name='nuclear_mask')

In [ ]:
#overwrite nuclear masks with one loaded in the napari viewer.
io.imsave(fov['nuclear_mask'],viewer.layers['nuclear_mask'].data)